In [2]:
import pandas as pd
import mysql.connector as msql

#Cargar el archivo CSV
df = pd.read_csv('extraidos.csv') # Indique el nombre del archivo csv
df = pd.DataFrame(df)

###############################
#Define la coneccion a la SQL
nomBDatos = 'cubo'        #Nombre de la base de datos 
nomTabla = 'intento'      #Nombre de la tabla de datos


###############################
#Crear querys para cargar los datos en el sercviodor
listacolumnas = list(df.columns)
strinCreateTabla = ''
for i in listacolumnas:
    tipoDato = df[i].dtype
    if tipoDato == 'O':
        tipo = ' CHAR (255) NOT NULL, '
    elif tipoDato == 'float64':
        tipo = ' FLOAT(2,1) NOT NULL ,'
    elif tipoDato == 'int64':
        tipo = ' INT(255) NOT NULL, '
    strinCreateTabla += str(i) + tipo
    
strinCreateTabla = strinCreateTabla[:-2]
# Query para crear la tabla
instruccionCT = "CREATE TABLE " + nomTabla + " (" + strinCreateTabla + ")"
    
#Query para crear la insertacion de datos 
elementoN = ['%s'] * (len(listacolumnas))
elementoN = ','.join(elementoN)
instruccionIC = "INSERT INTO " + nomBDatos + "." + nomTabla + " VALUES (" + elementoN + ")"


###############################
#Conexion de la base de datis
conn = msql.connect(
    host = "localhost", 
    port = 3306,
    database= nomBDatos,
    user= "root",
    password = ""
    )

#Si se conecta la base de datos cargo el archivo
if conn.is_connected():
    cursor = conn.cursor()
    cursor.execute("select database();")
    record = cursor.fetchone()
    print("SE A CONECTADO A LA BASE DE DATOS EN: ", record)
   
    queryT = 'DROP TABLE IF EXISTS ' + nomTabla +';'
    cursor.execute(queryT)
    cursor.execute(instruccionCT)
    print("Sea creado la tabla....")
    print(instruccionCT)
    
    #Cargar los datos de en la base de datos
    c = 0
    s = 0
    for i,row in df.iterrows():
        sql = instruccionIC
        cursor.execute(sql, tuple(row))
        
        if c >= 10000:
            s = s + c
            c = 0
            print("numero de datos " + str(s))
        c += 1
        conn.commit()



#Cierra la conexion con la base de datos
conn.close()

SE A CONECTADO A LA BASE DE DATOS EN:  ('cubo',)
Sea creado la tabla....
CREATE TABLE intento (SECTOR INT(255) NOT NULL, ESTADO CHAR (255) NOT NULL, FECHA CHAR (255) NOT NULL, ATENDIDOS INT(255) NOT NULL, GENERO CHAR (255) NOT NULL, CASO INT(255) NOT NULL, HOSPITALIZADOS INT(255) NOT NULL, DECESO INT(255) NOT NULL, UCI INT(255) NOT NULL, INTUBADO INT(255) NOT NULL)
10000
20000
30000
40000
50000
60000


KeyboardInterrupt: 